In [53]:
import pandas as pd
import numpy as np

In [54]:
dateList = pd.date_range(start='2004.12.01', end='2017.12.31')
base = pd.DataFrame(dateList, columns=['date'])
callRate = pd.read_csv('callRate.csv')

In [64]:
def month_ago(df):
    df['date'] = pd.to_datetime(df['date'])
    callRate = pd.merge(base, df, how='left')
    labeling = callRate.fillna(method='ffill')
    
    # 비교해줄 한달 전 df 생성
    compareDf= labeling.copy().rename(columns={'date':'compareDate', 'callRate':'compareRate'})
    labeling['compareDate']= labeling['date'] - pd.Timedelta(1,'M')
    # 병합을 위해 기준열의 type을 동일하게 맞춰줌
    print('type 변환 시작')
    for idx in range(len(labeling)):
        labeling['compareDate'][idx] = labeling['compareDate'][idx].strftime('%Y-%m-%d')
    print('type 변환 완료')
    
    moderator = pd.merge(labeling, compareDf, how='left')
    origin = moderator['callRate'].astype(float)
    compare = moderator['compareRate'].astype(float)

    label = (origin-compare)
    print('label 태깅 시작')
    for idx in range(31,len(label)):
        if label[idx] > 0:
            label[idx] = 'pos'
        elif label[idx] < 0:
            label[idx] = 'neg'
    print('label 태깅 완료')
    dfLabel = pd.concat([moderator['date'], label], axis=1).dropna()
    dfLabel.columns = ['date','label']
    dfLabel.to_csv('label.csv')
    # return dfLabel
    return dfLabel   

In [65]:
dfLabel=month_ago(callRate)

type 변환 시작


C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if sys.path[0] == '':


type 변환 완료
label 태깅 시작
label 태깅 완료


In [66]:
dfLabel['label'].value_counts()

pos    1940
neg    1901
0.0     907
Name: label, dtype: int64